# Working with Data

Data is the foundation on which machine learning models are built. Managing data centrally in the cloud, and making it accessible to teams of data scientists who are running experiments and training models on multiple workstations and compute targets is an important part of any professional data science solution.

In this lab, you'll explore two Azure Machine Learning objects for working with data: *Datastores*, and *Datasets*.

## Before You Start

Before you start this lab, ensure that you have completed the *Create an Azure Machine Learning Workspace* and *Create a Compute Instance* tasks in [Lab 1: Getting Started with Azure Machine Learning](./labdocs/Lab01.md). Then open this notebook in Jupyter on your Compute Instance.

## Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK.

> **Note**: If you do not have a current authenticated session with your Azure subscription, you'll be prompted to authenticate. Follow the instructions to authenticate using the code provided.

In [2]:
from azureml import core

ws = core.Workspace.from_config()
print(f'Ready to use Azure ML {core.VERSION} to work with {ws.name}')

Ready to use Azure ML 1.10.0 to work with workspace


## Work with a Datastore

In Azure ML, *datastores* are references to storage locations, such as Azure Storage blob containers. Every workspace has a default datastore - usually the Azure storage blob container that was created with the workspace. If you need to work with data that is stored in different locations, you can add custom datastores to your workspace and set any of them to be the default.

### View Datastores

Run the following code to determine the datastores in your workspace:

In [3]:
default_ds = ws.get_default_datastore()

for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

workspaceblobstore - Default = True
workspacefilestore - Default = False


You can also view and manage datastores in your workspace on the Datastores page for your workspace in [Azure ML Studio](https://ml.azure.com).

### Upload Data to a Datastore

Now that you have determined the available datastores, you can upload files from your local file system to a datastore so that it will be accessible to experiments running in the workspace, regardless of where the experiment script is actually being run.

In [4]:
default_ds.upload_files(
    files=['./data/diabetes.csv', './data/diabetes2.csv'],
    target_path='diabetes-data/',
    overwrite=True,
    show_progress=True
)

Uploading an estimated of 2 files
Uploading ./data/diabetes.csv
Uploading ./data/diabetes2.csv
Uploaded ./data/diabetes2.csv, 1 files out of an estimated total of 2
Uploaded ./data/diabetes.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_ed1d6934ab0e4886893abed93929c4d3

### Train a Model from a Datastore

When you uploaded the files in the code cell above, note that the code returned a *data reference*. A data reference provides a way to pass the path to a folder in a datastore to a script, regardless of where the script is being run, so that the script can access data in the datastore location.

The following code gets a reference to the **diabetes-data** folder where you uploaded the diabetes CSV files, and specifically configures the data reference for *download* - in other words, it can be used to download the contents of the folder to the compute context where the data reference is being used. Downloading data works well for small volumes of data that will be processed on local compute. When working with remote compute, you can also configure a data reference to *mount* the datastore location and read data directly from the data source.

> **More Information**: For more details about using datastores, see the [Azure ML documentation](https://docs.microsoft.com/azure/machine-learning/how-to-access-data).

In [5]:
data_ref = (
    default_ds.path('diabetes-data')
    .as_download(path_on_compute='diabetes_data')
)
print(data_ref)

$AZUREML_DATAREFERENCE_8792bec586514f24a3f90a759a593b34


The script will load the training data from the data reference passed to it as a parameter, so now you just need to set up the script parameters to pass the file reference when we run the experiment.

In [6]:
from azureml.train import sklearn
from azureml import widgets

script_params = {
    '--regularization': 0.1,
    '--data-folder': data_ref,
}


experiment_folder = 'diabetes-training-from-datastore'
config = sklearn.SKLearn(
    source_directory=experiment_folder,
    entry_script='diabetes_training.py',
    script_params=script_params,
    compute_target = 'local'
)

experiment_name = 'diabetes-training'
experiment = core.Experiment(workspace = ws, name = experiment_name)

run = experiment.submit(config=config)
widgets.RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1597272999_222bf2e2',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-08-12T22:56:44.513297Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'c2d956bc-22c3-4d55-8aed-56c265b9379d',
  'azureml.git.repository_uri': 'https://github.com/susumuasaga/mslearn-aml-labs',
  'mlflow.source.git.repoURL': 'https://github.com/susumuasaga/mslearn-aml-labs',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'fdae87a3b5a18ef10aa68b5d193ef96ba6217184',
  'mlflow.source.git.commit': 'fdae87a3b5a18ef10aa68b5d193ef96ba6217184',
  'azureml.git.dirty': 'False'},
 'inputDatasets': [],
 'runDefinition': {'script': 'diabetes_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments': ['--regularization',
   '0.1',
   '--data-folder',
   '$AZUREML_DATAREFERENCE_8792bec586514f24a3f90a759a593b34'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'commun

The first time the experiment is run, it may take some time to set up the Python environment - subsequent runs will be quicker.

When the experiment has completed, in the widget, view the **azureml-logs/70_driver_log.txt** output log to verify that the data files were downloaded before the experiment script was run.

## Work with Datasets

While you can read data directly from datastores, Azure Machine Learning provides a further abstraction for data in the form of *datasets*. A dataset is a versioned reference to a specific set of data that you may want to use in an experiment. Datasets can be *tabular* or *file*-based.

### Create a Tabular Dataset

Let's create a dataset from the diabetes data you uploaded to the datastore, and view the first 20 records. In this case, the data is in a structured format in a CSV file, so we'll use a *tabular* dataset.

In [8]:
default_ds = ws.get_default_datastore()

tab_data_set = core.Dataset.Tabular.from_delimited_files(
    path=(default_ds, 'diabetes-data/*.csv')
)

tab_data_set.take(20).to_pandas_dataframe()

PatientID  Pregnancies  PlasmaGlucose  DiastolicBloodPressure  \
0     1354778            0            171                      80   
1     1147438            8             92                      93   
2     1640031            7            115                      47   
3     1883350            9            103                      78   
4     1424119            1             85                      59   
5     1619297            0             82                      92   
6     1660149            0            133                      47   
7     1458769            0             67                      87   
8     1201647            8             80                      95   
9     1403912            1             72                      31   
10    1943830            1             88                      86   
11    1824483            3             94                      96   
12    1848869            5            114                     101   
13    1669231            7            110                      82   
14    1683688            0            148                      58   
15    1738587            3            109                      77   
16    1884264            3            106                      64   
17    1485251            1            156                      53   
18    1536832            8            117                      39   
19    1438701            3            102                     100   

    TricepsThickness  SerumInsulin        BMI  DiabetesPedigree  Age  Diabetic  
0                 34            23  43.509726          1.213191   21         0  
1                 47            36  21.240576          0.158365   23         0  
2                 52            35  41.511523          0.079019   23         0  
3                 25           304  29.582192          1.282870   43         1  
4                 27            35  42.604536          0.549542   22         0  
5                  9           253  19.724160          0.103424   26         0  
6                 19           227  21.941357          0.174160   21         0  
7                 43            36  18.277723          0.236165   26         0  
8                 33            24  26.624929          0.443947   53         1  
9                 40            42  36.889576          0.103944   26         0  
10                11            58  43.225041          0.230285   22         0  
11                31            36  21.294479          0.259020   23         0  
12                43            70  36.495320          0.079190   38         1  
13                16            44  36.089293          0.281276   25         0  
14                11           179  39.192076          0.160829   45         0  
15                46            61  19.847312          0.204345   21         1  
16                25            51  29.044573          0.589188   42         1  
17                15           226  29.786192          0.203824   41         1  
18                32           164  21.230996          0.089363   25         0  
19                25           289  42.185720          0.175593   43         1

As you can see in the code above, it's easy to convert a tabular dataset to a Pandas dataframe, enabling you to work with the data using common python techniques.

### Create a File Dataset

The dataset you created is a *tabular* dataset that can be read as a dataframe containing all of the data in the structured files that are included in the dataset definition. This works well for tabular data, but in some machine learning scenarios you might need to work with data that is unstructured; or you may simply want to handle reading the data from files in your own code. To accomplish this, you can use a *file* dataset, which creates a list of file paths in a virtual mount point, which you can use to read the data in the files.

In [9]:
file_data_set = core.Dataset.File.from_files(
    path=(default_ds, 'diabetes-data/*.csv')
)

for file_path in file_data_set.to_path():
    print(file_path)

/diabetes.csv
/diabetes2.csv


### Register Datasets

Now that you have created datasets that reference the diabetes data, you can register them to make them easily accessible to any experiment being run in the workspace.

We'll register the tabular dataset as **diabetes dataset**, and the file dataset as **diabetes files**.

In [10]:
tab_data_set = tab_data_set.register(
    workspace=ws, 
    name='diabetes dataset',
    description='diabetes data',
    tags = {'format':'CSV'},
    create_new_version=True
)

file_data_set = file_data_set.register(
    workspace=ws,
    name='diabetes file dataset',
    description='diabetes files',
    tags = {'format':'CSV'},
    create_new_version=True
)

print('Datasets registered')

Datasets registered


You can view and manage datasets on the **Datasets** page for your workspace in [Azure ML Studio](https://ml.azure.com). You cal also get a list of datasets from the workspace object:

In [7]:
print("Datasets:")
for dataset_name in ws.datasets.keys():
    dataset = core.Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

Datasets:
	 diabetes file dataset version 1
	 diabetes dataset version 1


The ability to version datasets enables you to redefine datasets without breaking existing experiments or pipelines that rely on previous definitions. By default, the latest version of a named dataset is returned, but you can retrieve a specific version of a dataset by specifying the version number, like this:

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


### Train a Model from a Tabular Dataset

Now that you have datasets, you're ready to start training models from them. You can pass datasets to scripts as *inputs* in the estimator being used to run the script.

Now you can create an estimator to run the script, and define a named *input* for the training dataset, which is read by the script.

> **Note**: The **Dataset** class is defined in the **azureml-dataprep** package (which is installed with the SDK), and this package includes optional support for **pandas** (which is used by the **to_pandas_dataframe()** method, so you need to include this package in the environment where the training experiment will be run.

In [8]:
from azureml.train import sklearn
from azureml import widgets

script_params = {
    '--regularization': 0.1
}

diabetes_ds = ws.datasets.get("diabetes dataset")

experiment_folder = 'diabetes-training-from-tab-dataset'
config = sklearn.SKLearn(
    source_directory=experiment_folder,
    entry_script='diabetes_training.py',
    script_params=script_params,
    compute_target = 'local',
    inputs=[diabetes_ds.as_named_input('diabetes')],
    pip_packages=['azureml-dataprep[pandas]'],
)

experiment_name = 'diabetes-training'
experiment = core.Experiment(workspace = ws, name = experiment_name)

run = experiment.submit(config=config)
widgets.RunDetails(run).show()
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1597282996_c5e0aae1',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-08-13T01:45:15.795354Z',
 'endTimeUtc': '2020-08-13T01:45:34.561908Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '8b0be9e9-1fe0-4e83-9016-a49de62e5c0f',
  'azureml.git.repository_uri': 'https://github.com/susumuasaga/mslearn-aml-labs',
  'mlflow.source.git.repoURL': 'https://github.com/susumuasaga/mslearn-aml-labs',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'fdae87a3b5a18ef10aa68b5d193ef96ba6217184',
  'mlflow.source.git.commit': 'fdae87a3b5a18ef10aa68b5d193ef96ba6217184',
  'azureml.git.dirty': 'False'},
 'inputDatasets': [{'dataset': {'id': 'ae295d27-1d6d-4897-b7d2-d19f9275b922'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'diabetes', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'diabetes_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  '

The first time the experiment is run, it may take some time to set up the Python environment - subsequent runs will be quicker.

When the experiment has completed, in the widget, view the **azureml-logs/70_driver_log.txt** output log and the metrics generated by the run.

### Train a Model from a File Dataset

You've seen how to train a model using training data in a *tabular* dataset; but what about a *file* dataset?

When you're using a file dataset, the dataset input passed to the script represents a mount point containing file paths. How you read the data from these files depends on the kind of data in the files and what you want to do with it. In the case of the diabetes CSV files, you can use the Python **glob** module to create a list of files in the virtual mount point defined by the dataset, and read them all into Pandas dataframes that are concatenated into a single dataframe.

Next we need to change the way we pass the dataset to the estimator - it needs to define a mount point from which the script can read the files. For large volumes of data, you'd generally use the **as_mount** method to stream the files directly from the dataset source; but when running on local compute (as we are in this example), you need to use the **as_download** option to download the dataset files to a local folder.

Also, since the **Dataset** class is defined in the **azureml-dataprep** package, we need to include that in the experiment environment.

In [13]:
from azureml import widgets

script_params = {
    '--regularization': 0.1
}

diabetes_ds = ws.datasets.get("diabetes file dataset")

experiment_folder = 'diabetes-training-from-file-dataset'
config = sklearn.SKLearn(
    source_directory=experiment_folder,
    entry_script='diabetes_training.py',
    script_params=script_params,
    compute_target = 'local',
    inputs=[
        diabetes_ds.as_named_input('diabetes')
        .as_download(path_on_compute='diabetes_data')
    ],
    pip_packages=['azureml-dataprep[pandas]']
)

experiment_name = 'diabetes-training'
experiment = core.Experiment(workspace = ws, name = experiment_name)

run = experiment.submit(config=config)
widgets.RunDetails(run).show()
run.wait_for_completion()


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1597286153_ded6e4ab',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-08-13T02:35:54.84107Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '687012f8-8f73-40a2-976b-8ad1f1f31528',
  'azureml.git.repository_uri': 'https://github.com/susumuasaga/mslearn-aml-labs',
  'mlflow.source.git.repoURL': 'https://github.com/susumuasaga/mslearn-aml-labs',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '8309a54b38391169b196652b65330ff25eb62d02',
  'mlflow.source.git.commit': '8309a54b38391169b196652b65330ff25eb62d02',
  'azureml.git.dirty': 'False'},
 'inputDatasets': [{'dataset': {'id': '277ef6e5-d13f-40e8-add5-e087e820ac73'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'diabetes', 'mechanism': 'Download', 'pathOnCompute': 'diabetes_data'}}],
 'runDefinition': {'script': 'diabetes_training.py',
  'scriptType': None,
  'useAbsolutePath': False,
  'arguments'

When the experiment has completed, in the widget, view the **azureml-logs/70_driver_log.txt** output log to verify that the file dataset was processed and the data files downloaded.